In [ ]:
Timeline choropleth - Incomplete

2021-07-20 <br/>
US Droughts <br/>
https://github.com/rfordatascience/tidytuesday/blob/master/data/2021/2021-07-20/readme.md

https://www.kaggle.com/balagpdy/heatmap-animation-us-drought-map

In [3]:
import pandas as pd 

In [4]:
df=pd.read_csv('drought.csv')

In [5]:
df.head()

,map_date,state_abb,valid_start,valid_end,stat_fmt,drought_lvl,area_pct,area_total,pop_pct,pop_total
0,20210713,AK,2021-07-13,2021-07-19,1,None,74.35,433133.18,33.91,240644.16
1,20210713,AK,2021-07-13,2021-07-19,1,D0,25.65,149435.11,66.09,468985.84
2,20210713,AK,2021-07-13,2021-07-19,1,D1,0.00,0.00,0.00,0.00
3,20210713,AK,2021-07-13,2021-07-19,1,D2,0.00,0.00,0.00,0.00
4,20210713,AK,2021-07-13,2021-07-19,1,D3,0.00,0.00,0.00,0.00


In [8]:
def convert_numeric(x):
    y=None
    if x=='D0':
        y=1
    if x=='D1':
        y=2
    if x=='D2':
        y=3
    if x=='D3':
        y=4
    


In [9]:
a=df[df.valid_start=='2021-07-13']

In [10]:
a.shape

(312, 10)

In [20]:
#########################
### PART 1: DATA HANDLING
#########################

import pandas as pd
import numpy as np

### Read in county data and drought info data
county = pd.read_csv('county_info_2016.csv', encoding = "ISO-8859-1")
county.columns = ['USPS','GEOID','ANSICODE','NAME','ALAND','AWATER','ALAND_SQMI','AWATER_SQMI','INTPTLAT','INTPTLONG' ]
county = county[['GEOID','ALAND_SQMI','INTPTLAT','INTPTLONG']]

dr = pd.read_csv('drought.csv')

### Check if there are NaNs and all entries are recorded week's internal
#dr.isnull().sum() ## To check if there are NaNs
#dr.validStart = pd.to_datetime(dr.validStart, format='%Y-%m-%d')
#dr.validEnd = pd.to_datetime(dr.validEnd, format='%Y-%m-%d')
#((dr.validEnd - dr.validStart)!="6 day").sum()
#dr.releaseDate = pd.to_datetime(dr.releaseDate, format='%Y-%m-%d')
#(dr.releaseDate != dr.validStart).sum()



In [21]:
dr.head()

,map_date,state_abb,valid_start,valid_end,stat_fmt,drought_lvl,area_pct,area_total,pop_pct,pop_total
0,20210713,AK,2021-07-13,2021-07-19,1,None,74.35,433133.18,33.91,240644.16
1,20210713,AK,2021-07-13,2021-07-19,1,D0,25.65,149435.11,66.09,468985.84
2,20210713,AK,2021-07-13,2021-07-19,1,D1,0.00,0.00,0.00,0.00
3,20210713,AK,2021-07-13,2021-07-19,1,D2,0.00,0.00,0.00,0.00
4,20210713,AK,2021-07-13,2021-07-19,1,D3,0.00,0.00,0.00,0.00


In [22]:
### Data cleansing
dr['releaseDate'] = pd.to_datetime(dr.map_date, format='%Y-%m-%d')
num_m = len(dr.releaseDate.unique())
dr = dr.drop(dr[[ 'valid_start', 'valid_end', 'stat_fmt']], axis=1)

### Resample data on monthly basis
dr_m = dr.set_index('releaseDate').groupby(['state_abb']).resample('M').mean()

# ### Calculate drought level (when NONE is 100% => 0, if D4 is 100% => 5, and linearly between 0 and 5)
# dr_m['LEVEL'] = (dr_m.D4*5 + (dr_m.D3-dr_m.D4)*4 + (dr_m.D2-dr_m.D3)*3 + (dr_m.D1-dr_m.D2)*2 + (dr_m.D0-dr_m.D1))/100

### Merge drought data and county info (coordination, size) by FIPS/GEOID
dr_m = dr_m.reset_index(level=1)
# dr_final = pd.merge(dr_m, county, left_on='FIPS', right_on='GEOID', how='inner', sort='False')
# dr_final = dr_final[['FIPS', 'releaseDate', 'LEVEL', 'ALAND_SQMI', 'INTPTLAT', 'INTPTLONG']]
# dr_final = dr_final.groupby('FIPS')



In [23]:
dr_m.head()

,releaseDate,map_date,area_pct,area_total,pop_pct,pop_total
state_abb,,,,,,
AK,1970-01-31,2.011103e+07,16.898282,98444.040187,17.195279,1.220230e+05
AL,1970-01-31,2.011103e+07,22.967267,11833.931803,23.201132,1.104578e+06
AR,1970-01-31,2.011103e+07,21.489162,11456.926458,21.369599,6.260091e+05
AZ,1970-01-31,2.011103e+07,36.323092,42214.167834,35.493812,2.333638e+06
CA,1970-01-31,2.011103e+07,33.031783,52308.152705,33.586630,1.243335e+07


In [ ]:

#####################################
### PART 2: HEATMAP ANIMATION DISPLAY
#####################################

import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from mpl_toolkits.basemap import Basemap
from matplotlib.animation import FuncAnimation
#get_ipython().magic('matplotlib nbagg') ## Only for Jupyter iPython display

### Create a figure and draw a basemap
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
m = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
m.drawcoastlines() #zorder=3
m.drawmapboundary(zorder=0) #fill_color='#9fdbff'
m.fillcontinents(color='#ffffff',zorder=1) #,lake_color='#9fdbff',alpha=1
m.drawcountries(linewidth=1.5) #color='darkblue'
m.drawstates() #zorder=3

### Set county location values, drough level values, marker sizes (according to county size), colormap and title 
x, y = m(dr_final.nth(0).INTPTLONG.tolist(), dr_final.nth(0).INTPTLAT.tolist())
colors = (dr_final.nth(0).LEVEL).tolist()
sizes = (dr_final.nth(0).ALAND_SQMI/7.5).tolist()
cmap = plt.cm.YlOrRd
sm = ScalarMappable(cmap=cmap)
plt.title('US Drought Level (Year-Month): '+dr_final.nth(0).releaseDate.iloc[0].strftime('%Y-%m'))

### Display the scatter plot and its colorbar (0-5)
scatter = ax.scatter(x,y,s=sizes,c=colors,cmap=cmap,alpha=1,edgecolors='face',marker='H',vmax=5,vmin=0,zorder=1.5)
plt.colorbar(scatter)

## Update function for animation
def update(ii):
    colors = (dr_final.nth(ii).LEVEL).tolist()
    scatter.set_color(sm.to_rgba(colors))
    plt.title('US Drought Level (Year-Month): '+dr_final.nth(ii).releaseDate.iloc[0].strftime('%Y-%m'))
    return scatter, 

anim = FuncAnimation(plt.gcf(),update,interval=300,repeat=False,frames=203,blit=True) #blit=True
anim.save('Heatmap_animation_US_Drought.gif', writer='imagemagick')
#plt.show()

###################################
### PART 3: ANIMATION SAVING AS MP4
###################################

### Save functions for animation 
#from matplotlib import rc, animation
#mywriter = animation.FFMpegWriter()
#anim.save('Heatmap_animation_US_Drought.mp4',writer=mywriter)

In [44]:
#########################3#
### Written by Jaeyoon Park 
### 16 Jan 2017
### Python 3.5
###########################

#########################
### PART 1: DATA HANDLING
#########################

import pandas as pd
import numpy as np

### Read in county data and drought info data
county = pd.read_csv('county_info_2016.csv', encoding = "ISO-8859-1")
county.columns = ['USPS','GEOID','ANSICODE','NAME','ALAND','AWATER','ALAND_SQMI','AWATER_SQMI','INTPTLAT','INTPTLONG' ]
county = county[['GEOID','ALAND_SQMI','INTPTLAT','INTPTLONG']]

dr = pd.read_csv('us-droughts.csv')

### Check if there are NaNs and all entries are recorded week's internal
#dr.isnull().sum() ## To check if there are NaNs
#dr.validStart = pd.to_datetime(dr.validStart, format='%Y-%m-%d')
#dr.validEnd = pd.to_datetime(dr.validEnd, format='%Y-%m-%d')
#((dr.validEnd - dr.validStart)!="6 day").sum()
#dr.releaseDate = pd.to_datetime(dr.releaseDate, format='%Y-%m-%d')
#(dr.releaseDate != dr.validStart).sum()

### Data cleansing
dr.releaseDate = pd.to_datetime(dr.releaseDate, format='%Y-%m-%d')
num_m = len(dr.releaseDate.unique())
dr = dr.drop(dr[['county', 'state', 'validStart', 'validEnd', 'domStatisticFormatID']], axis=1)

### Resample data on monthly basis
dr_m = dr.set_index('releaseDate').groupby(['FIPS']).resample('M').mean()

### Calculate drought level (when NONE is 100% => 0, if D4 is 100% => 5, and linearly between 0 and 5)
dr_m['LEVEL'] = (dr_m.D4*5 + (dr_m.D3-dr_m.D4)*4 + (dr_m.D2-dr_m.D3)*3 + (dr_m.D1-dr_m.D2)*2 + (dr_m.D0-dr_m.D1))/100

dr_m = dr_m.drop(dr_m[['FIPS']], axis=1)

### Merge drought data and county info (coordination, size) by FIPS/GEOID
dr_m = dr_m.reset_index()
dr_final = pd.merge(dr_m, county, left_on='FIPS', right_on='GEOID', how='inner', sort='False')
dr_final = dr_final[['FIPS', 'releaseDate', 'LEVEL', 'ALAND_SQMI', 'INTPTLAT', 'INTPTLONG']]
dr_final = dr_final.groupby('FIPS')



In [48]:
dr_final.head()

,FIPS,releaseDate,LEVEL,ALAND_SQMI,INTPTLAT,INTPTLONG
0,1001,2000-01-31,1.75000,594.446,32.532237,-86.646440
1,1001,2000-02-29,1.00000,594.446,32.532237,-86.646440
2,1001,2000-03-31,1.31855,594.446,32.532237,-86.646440
3,1001,2000-04-30,0.98225,594.446,32.532237,-86.646440
4,1001,2000-05-31,1.83296,594.446,32.532237,-86.646440
...,...,...,...,...,...,...
652036,72153,2000-01-31,0.00000,67.711,18.085669,-66.857901
652037,72153,2000-02-29,0.00000,67.711,18.085669,-66.857901
652038,72153,2000-03-31,0.00000,67.711,18.085669,-66.857901
652039,72153,2000-04-30,0.00000,67.711,18.085669,-66.857901


In [49]:
!pip install https://github.com/matplotlib/basemap/archive/master.zip

    ERROR: Command errored out with exit status 1:
     command: 'd:\users\dcomputer\desktop\learning\streamlined eda and ml\project_env\scripts\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\DCOMPU~1\\AppData\\Local\\Temp\\pip-req-build-jjhdr8tr\\setup.py'"'"'; __file__='"'"'C:\\Users\\DCOMPU~1\\AppData\\Local\\Temp\\pip-req-build-jjhdr8tr\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\DCOMPU~1\AppData\Local\Temp\pip-pip-egg-info-zi4vk5sb'
         cwd: C:\Users\DCOMPU~1\AppData\Local\Temp\pip-req-build-jjhdr8tr\
    Complete output (18 lines):
    checking for GEOS lib in C:\Users\DComputer ....
    checking for GEOS lib in C:\Users\DComputer\local ....
    checking for GEOS lib in /usr ....
    checking for GEOS lib in /usr/local ....
    checking for GEOS lib in /sw ....
    checking for GEOS 

In [50]:

#####################################
### PART 2: HEATMAP ANIMATION DISPLAY
#####################################

import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from mpl_toolkits.basemap import Basemap
from matplotlib.animation import FuncAnimation
#get_ipython().magic('matplotlib nbagg') ## Only for Jupyter iPython display

### Create a figure and draw a basemap
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
m = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
m.drawcoastlines() #zorder=3
m.drawmapboundary(zorder=0) #fill_color='#9fdbff'
m.fillcontinents(color='#ffffff',zorder=1) #,lake_color='#9fdbff',alpha=1
m.drawcountries(linewidth=1.5) #color='darkblue'
m.drawstates() #zorder=3

### Set county location values, drough level values, marker sizes (according to county size), colormap and title 
x, y = m(dr_final.nth(0).INTPTLONG.tolist(), dr_final.nth(0).INTPTLAT.tolist())
colors = (dr_final.nth(0).LEVEL).tolist()
sizes = (dr_final.nth(0).ALAND_SQMI/7.5).tolist()
cmap = plt.cm.YlOrRd
sm = ScalarMappable(cmap=cmap)
plt.title('US Drought Level (Year-Month): '+dr_final.nth(0).releaseDate.iloc[0].strftime('%Y-%m'))

### Display the scatter plot and its colorbar (0-5)
scatter = ax.scatter(x,y,s=sizes,c=colors,cmap=cmap,alpha=1,edgecolors='face',marker='H',vmax=5,vmin=0,zorder=1.5)
plt.colorbar(scatter)

## Update function for animation
def update(ii):
    colors = (dr_final.nth(ii).LEVEL).tolist()
    scatter.set_color(sm.to_rgba(colors))
    plt.title('US Drought Level (Year-Month): '+dr_final.nth(ii).releaseDate.iloc[0].strftime('%Y-%m'))
    return scatter, 

anim = FuncAnimation(plt.gcf(),update,interval=300,repeat=False,frames=203,blit=True) #blit=True
anim.save('Heatmap_animation_US_Drought.gif', writer='imagemagick')
#plt.show()

###################################
### PART 3: ANIMATION SAVING AS MP4
###################################

### Save functions for animation 
#from matplotlib import rc, animation
#mywriter = animation.FFMpegWriter()
#anim.save('Heatmap_animation_US_Drought.mp4',writer=mywriter)

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'